In [1]:
import torch
import random
import parameters
import time
import preprocess
import similarities
import pandas as pd
import numpy as np
import utilities
import pandas as pd
import numpy as np
import preprocess
import utilities
import seaborn as sns
from numpy import dot
from numpy.linalg import norm
from itertools import combinations
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
paths = parameters.data_paths
df = utilities.read_data(paths['ohsumed'])
df = df.sample(5000)
X = df['text']
y = df.drop(['text'], axis=1)
X = X.apply(preprocess.preprocess_text)

In [3]:
model_name = 'distiluse-base-multilingual-cased-v1'

In [4]:
X_num = utilities.vectorize_data(X, model_name)

In [5]:
text = X

In [6]:
data_paths = parameters.data_paths
sim_calculation_type = parameters.sim_calculation_type
all_sentence_embeddings =  parameters.huggingface_embeddings + parameters.openai_embeddings + parameters.google_embeddings

In [7]:
def calculate_similarity_matrix(X, y, sim_method='cosine'):
    
    import similarities
    import time
    
    sim_df = pd.DataFrame(index=y.columns, columns=y.columns)
    t3 = time.time()
    for col in y.columns:
        indexes = y[y[col]==1].index
        sim_df.loc[col, col] = similarities.calculate_similarity_within_classes(X.loc[indexes])
    t4 = time.time()
    for col1, col2 in list(combinations(y.columns, 2)):
        sim_df.loc[col1, col2] = similarities.calculate_similarity_between_classes(X[y[y[col1]==1].index], X[y[y[col2]==1].index])
    
    return sim_df, t3, t4

In [8]:
def calculate_within_class(X, y, sim_calculation_type, sim_df):
    
    t1 = time.time()
    import ray
    ray.init()
    
    @ray.remote
    def run(vecs, sim_calculation_type):
        return similarities.calculate_similarity_within_classes(vecs, sim_calculation_type)

    futures = [run.remote(X.loc[y[y[col]==1].index], sim_calculation_type) for col in y.columns]
    results = ray.get(futures) 
    
    for col, sim in zip(y.columns, results):    
        sim_df.loc[col, col] = sim
    
    ray.shutdown()
    print('within class ', time.time() - t1)
    return sim_df

In [9]:
def calculate_between_class(X, y, sim_df):
    
    t1 = time.time()
    import ray
    ray.init()
    
    @ray.remote
    def run(vecs1, vecs2):
        return similarities.calculate_similarity_between_classes(vecs1, vecs2)
    
    futures = [run.remote(X[y[y[col1]==1].index], X[y[y[col2]==1].index]) for col1, col2 in list(combinations(y.columns, 2))]
    results = ray.get(futures) 
    
    for (col1, col2), sim in zip(list(combinations(y.columns, 2)), results):
        sim_df.loc[col1, col2] = sim
    
    ray.shutdown()
    print('between class ', time.time() - t1)
    return sim_df

In [10]:
def calculate_similarity_matrix(X, y, sim_method='cosine'):
    
    import similarities
    
    sim_df = pd.DataFrame(index=y.columns, columns=y.columns)
    
    sim_df = calculate_within_class(X, y, sim_calculation_type, sim_df)
    
    t4 = time.time()    
    
    sim_df = calculate_between_class(X, y, sim_df)
    
    return sim_df, t3, t4

In [11]:
%%time
ss1 = calculate_similarity_matrix(X_num, y)

CPU times: total: 1min 55s
Wall time: 1min 55s


In [16]:
%%time
ss2 = calculate_similarity_matrix2(X_num, y)

within class  11.716727495193481
between class  52.255552768707275
CPU times: total: 3.95 s
Wall time: 1min 3s


In [13]:
asda

NameError: name 'asda' is not defined

In [17]:
ss2[0]

,c01,c02,c05,c06,c08,c10,c11,c12,c13,c15,c16,c17,c18,c20,c21
c01,0.332783,0.319417,0.290775,0.318332,0.308782,0.280624,0.287214,0.298117,0.281763,0.310628,0.279526,0.294399,0.267007,0.309315,0.27288
c02,NaN,0.353181,0.264831,0.294561,0.291214,0.271293,0.27386,0.280645,0.272244,0.304072,0.264524,0.28143,0.258796,0.322321,0.255097
c05,NaN,NaN,0.370809,0.306416,0.289918,0.320822,0.309081,0.303307,0.276087,0.310587,0.301303,0.303868,0.27805,0.276071,0.307217
c06,NaN,NaN,NaN,0.34463,0.303158,0.291694,0.298201,0.321356,0.286849,0.319223,0.290828,0.293861,0.293065,0.293279,0.284471
c08,NaN,NaN,NaN,NaN,0.372401,0.289816,0.282052,0.284775,0.271883,0.30631,0.301308,0.277891,0.255432,0.301452,0.278444
c10,NaN,NaN,NaN,NaN,NaN,0.329362,0.301411,0.291705,0.267595,0.300627,0.296934,0.286379,0.279179,0.276882,0.287738
c11,NaN,NaN,NaN,NaN,NaN,NaN,0.365357,0.291939,0.267133,0.302649,0.290081,0.291506,0.266855,0.276007,0.277223
c12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.355355,0.301128,0.317075,0.28885,0.288173,0.300486,0.282312,0.271567
c13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.347535,0.290977,0.300924,0.262792,0.262709,0.266546,0.251898
c15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.350982,0.301713,0.306897,0.29842,0.312176,0.273494


In [15]:
ss1[0]

,c01,c02,c05,c06,c08,c10,c11,c12,c13,c15,c16,c17,c18,c20,c21
c01,0.332783,0.319417,0.290775,0.318332,0.308782,0.280624,0.287214,0.298117,0.281763,0.310628,0.279526,0.294399,0.267007,0.309315,0.27288
c02,NaN,0.353181,0.264831,0.294561,0.291214,0.271293,0.27386,0.280645,0.272244,0.304072,0.264524,0.28143,0.258796,0.322321,0.255097
c05,NaN,NaN,0.370809,0.306416,0.289918,0.320822,0.309081,0.303307,0.276087,0.310587,0.301303,0.303868,0.27805,0.276071,0.307217
c06,NaN,NaN,NaN,0.34463,0.303158,0.291694,0.298201,0.321356,0.286849,0.319223,0.290828,0.293861,0.293065,0.293279,0.284471
c08,NaN,NaN,NaN,NaN,0.372401,0.289816,0.282052,0.284775,0.271883,0.30631,0.301308,0.277891,0.255432,0.301452,0.278444
c10,NaN,NaN,NaN,NaN,NaN,0.329362,0.301411,0.291705,0.267595,0.300627,0.296934,0.286379,0.279179,0.276882,0.287738
c11,NaN,NaN,NaN,NaN,NaN,NaN,0.365357,0.291939,0.267133,0.302649,0.290081,0.291506,0.266855,0.276007,0.277223
c12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.355355,0.301128,0.317075,0.28885,0.288173,0.300486,0.282312,0.271567
c13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.347535,0.290977,0.300924,0.262792,0.262709,0.266546,0.251898
c15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.350982,0.301713,0.306897,0.29842,0.312176,0.273494


In [ ]:
ss1

In [ ]:
t2 = time.time()
sim_df, t3, t4 = calculate_similarity_matrix(X_num, y)
t5 = time.time()
print('total :',(t5-t2)/60, 'between_class: ',(t5-t4)/60, 'within class: ',(t4-t3)/60)

In [ ]:
t2 = time.time()
sim_df2, t3, t4 = calculate_similarity_matrix2(X_num, y)
t5 = time.time()
print('total :',(t5-t2)/60, 'between_class: ',(t5-t4)/60, 'within class: ',(t4-t3)/60)

In [ ]:
stop

In [ ]:
class1_indexes = y[y['c01']==1].index[0:10]


In [ ]:
class1_indexes = y[]

In [18]:
aa = X_num.copy()

In [19]:
X_num = X_num.iloc[0:20].apply(lambda x: x[0:10]).reset_index(drop=True)

In [34]:
np.random.seed(1)

In [76]:
sample_in_each_class = 1000
X_num = pd.Series([np.random.random(10) for i in range(2*sample_in_each_class)])
class1_indexes = list(range(sample_in_each_class))
class2_indexes = list(range(sample_in_each_class,2*sample_in_each_class))

In [77]:
%%time
def cosine_similarity(vec1, vec2):
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    if norm1 == 0:
        norm1 += 0.00001
    if norm2 == 0:
        norm2 += 0.00001  
    return np.dot(vec1, vec2)/(norm1*norm2)
    
approach1 = []
for idx1 in class1_indexes:
    for idx2 in class2_indexes:
        approach1.append(cosine_similarity(X_num.loc[idx1], X_num.loc[idx2]))
np.mean(approach1), len(approach1)

CPU times: total: 25.4 s
Wall time: 25.2 s


(0.7541128841787165, 1000000)

In [78]:
%%time
import itertools
vectors_product = itertools.product(X_num[class1_indexes], X_num[class2_indexes])
vectors_product = pd.Series(list(vectors_product))
approach2 = vectors_product.apply(lambda x: cosine_similarity(x[0], x[1]))
np.mean(approach2), len(approach2)

CPU times: total: 9.23 s
Wall time: 9.23 s


(0.7541128841786969, 1000000)

In [79]:
%%time
vectors_product = itertools.product(X_num[class1_indexes], X_num[class2_indexes])
vectors_product = np.array(list(vectors_product))
first_part = vectors_product[:,0,:]
second_part = vectors_product[:,1,:]

numerator = np.sum(np.multiply(first_part, second_part), axis=1)
denominator = (np.multiply(np.linalg.norm(first_part, axis=1), 
                           np.linalg.norm(second_part, axis=1)))
approach3 = numerator / denominator
np.mean(approach3), len(approach3)

CPU times: total: 1.17 s
Wall time: 1.16 s


(0.7541128841786969, 1000000)

In [75]:
np.mean(approach3), len(approach3)

(0.7597549062958892, 1000000)

In [80]:
25.2/1.16

21.724137931034484

In [ ]:
import itertools

In [ ]:
%%time
normal_way = []
for idx1 in class1.index:
    for idx2 in class2.index:
        normal_way.append(cosine_similarity(X_num.loc[idx1], X_num.loc[idx2]))

In [ ]:
%%time
vectors_product = pd.Series(list(itertools.product(X_num[class1_indexes], X_num[class2_indexes])))
way2 = vectors_product.apply(lambda x: cosine_similarity(x[0], x[1]))

In [ ]:
%%time
vectors_product = np.array(list(itertools.product(X[y[y[col1]==1].index], X[y[y[col2]==1].index])))
first_part = vectors_product[:,0,:]
second_part = vectors_product[:,1,:]
way3 = np.sum(np.multiply(first_part, second_part), axis=1)/(np.multiply(np.linalg.norm(first_part, axis=1), np.linalg.norm(second_part, axis=1)))

In [ ]:
def calculate_between_class_similarities(col1, col2, X, y):
    
    vectors_product = np.array(list(product(X[y[y[col1]==1].index], X[y[y[col2]==1].index])))
    first_part = vectors_product[:,0,:]
    second_part = vectors_product[:,1,:]
    sims = np.sum(np.multiply(first_part, second_part), axis=1)/(np.multiply(np.linalg.norm(first_part, axis=1), np.linalg.norm(second_part, axis=1)))
    
    return np.mean(sims)   

In [ ]:
(y[y['c01']==1]).index

In [ ]:
sum(way3)/len(way3), np.mean(way3)

In [ ]:
from numpy.linalg import norm as n

In [ ]:
A = first_part
B = second_part
way4 = 1.0 - np.dot(A/n(A,axis=1)[:,None],(B/n(B,axis=1)[:,None]).T)

In [ ]:
np.matmul(first_part, second_part, out=w)

In [ ]:
first_part.shape

In [ ]:
first_part.shape

In [ ]:
w = np.random.rand(1274446)
w

In [ ]:
jj = 

In [ ]:
np.sum(np.multiply(first_part, second_part), axis=1)

In [ ]:
first_part = aa[:,0,:]
second_part = aa[:,1,:]
first_part.shape, second_part.shape

In [ ]:
cc1 = np.linalg.norm(first_part, axis=1)
cc2 = np.linalg.norm(second_part, axis=1)

In [ ]:
cc1[2]*cc2[2]

In [ ]:
np.multiply(np.linalg.norm(first_part, axis=1), np.linalg.norm(second_part, axis=1)).shape

In [ ]:
np.dot(first_part, second_part, out=first_part).shape

In [ ]:
np.linalg.norm(first_part, axis=1)

In [ ]:
kk = np.array(aa)

In [ ]:
kk[0].shape

In [ ]:
%%time
for i in range(10000):
    for j in X_num[y[y['c11']==1].index]:
        np.linalg.norm(j)

In [ ]:
X_num[y[y['c11']==1].index]

In [ ]:
X_num[y[y['c11']==1].index].apply(lambda x: np.linalg.norm(x))

In [ ]:
itertools

In [ ]:
X_num[y[y['c11']==1].index], X_num[y[y['c15']==1].index]

In [ ]:
vecs

In [ ]:
X_num.loc[idx1].shape

In [ ]:
len(y[y['c11']==1].index)*len(y[y['c15']==1].index)